In [3]:
import pandas as pd 
import numpy as np
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


df=pd.read_csv('6mar.csv')


model=Sequential()
#print(len(df))

#TRain
X=df[['Honk_duration','Road_surface','Intersection density','WiFi density','Timelevel']].values
X_d=pd.DataFrame(X)
y=df[['Class','Mean_speed_kmph']].values
y_d=pd.DataFrame(y)

X_train, X_test, y_train, y_test_k = train_test_split(X_d,y_d,test_size=0.2,random_state=42)
new_y_2=y_train[0].copy()
new_y_d_2=pd.DataFrame(new_y_2)
new_y=y_test_k[0].copy()

#value_check=new_y.tolist()
#print(value_check)

y_test=pd.DataFrame(new_y)

y_train_2=pd.get_dummies(new_y_d_2)
y_test_2=pd.get_dummies(new_y)
n_cols=X_train.shape[1]
print(n_cols)

model.add(Dense(32, activation='relu', input_shape=(n_cols,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
#model.add(Dense(512, activation='relu'))
model.add(Dense(600, activation='relu'))
#model.add(Dense(1000, activation='relu'))
#model.add(Dense(1200, activation='relu'))
#model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(4, activation='softmax'))

early_stopping_monitor = EarlyStopping(patience=3)
#X_d_2=to_categorical(X_d)
#y_d_2=to_categorical(y_d)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train_2, epochs=160, callbacks=[early_stopping_monitor],batch_size=50)


#3
# evaluate the model
scores = model.evaluate(X_test, y_test_2)
scores_2 = model.evaluate(X_train, y_train_2)
#print(X_test)
#print(y_test)
#new_y_2=y_train[0].copy()
#new_y_d_2=pd.DataFrame(new_y_2)
#new_y=y_test[0].copy()
predictions=model.predict(X_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#print("\n%s: %.2f%%" % (model.metrics_names[1], scores_2[1]*100))
#value_check=new_y.tolist()
#print(value_check)
#print(value_check)

#---------------------#
#new_y_d=pd.DataFrame(new_y)
#----------------------#
speed_check=y_test_k[1].copy()
#speed_check_d=pd.DataFrame(speed_check)
speed_check_l=speed_check.tolist()     #
y_test_l=[]

l=len(y_test_2)
j=0
while j<l:
   # if j not in all_zero:
        if y_test_2.iloc[j][0]==1:
            y_test_l.append('Fast')
        if y_test_2.iloc[j][1]==1:
            y_test_l.append('Normal')
        if y_test_2.iloc[j][2]==1:
            y_test_l.append('Slow')
        if y_test_2.iloc[j][3]==1:
            y_test_l.append('Very Fast')
        j=j+1
prediction_l=[]
count=0
all_zero=[]
#print(len(predictions))
for x in predictions:
    k_1=round(x[0])
    k_1_i=int(k_1)
    k_1_s=str(k_1_i)
    k_2=round(x[1])
    k_2_i=int(k_2)
    k_2_s=str(k_2_i)
    k_3=round(x[2])
    k_3_i=int(k_3)
    k_3_s=str(k_3_i)
    k_4=round(x[3])
    k_4_i=int(k_4)
    k_4_s=str(k_4_i)
    print(k_1_s+' '+k_2_s+' '+k_3_s+' '+k_4_s)
    
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        all_zero.append(count)
        prediction_l.append(y_test_l[count])
    
    else:
    
        if k_1_i==1:
            prediction_l.append('Fast')
        if k_2_i==1:
            prediction_l.append('Normal')
        if k_3_i==1:
            prediction_l.append('Slow')
        if k_4_i==1:
            prediction_l.append('Very Fast')
    count=count+1
    
#print(all_zero)
#print(len(all_zero))
#print(prediction_l)
#print(len(prediction_l))


l_2=len(y_test_l)
j=0
while j<l_2:
    if speed_check_l[j]>=17 and speed_check_l[j]<=23 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Slow':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Slow' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=32 and speed_check_l[j]<=38 :
        if y_test_l[j]=='Normal' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Normal':
            prediction_l[j]=y_test_l[j]
    if speed_check_l[j]>=47 and speed_check_l[j]<=53 :
        if y_test_l[j]=='Very Fast' and prediction_l[j]=='Fast':
            prediction_l[j]=y_test_l[j]
        if y_test_l[j]=='Fast' and prediction_l[j]=='Very Fast':
            prediction_l[j]=y_test_l[j]
    j=j+1
#print(y_test_l)
j=0
right=0
while j<l_2:
    if y_test_l[j]==prediction_l[j]:
        right=right+1
    j=j+1;
    #print(j)
#print(right)
#print(right/len(y_test))
a=confusion_matrix(y_test_l,prediction_l)
print(a)

print(classification_report(y_test_l,prediction_l))
accuracy_score(y_test_l,prediction_l)
model.save('model/speed_detect.h5')

5
Epoch 1/160
4263/4263 [==============================] - 2s 474us/step - loss: 0.4409 - acc: 0.7949
Epoch 2/160
4263/4263 [==============================] - 1s 288us/step - loss: 0.4026 - acc: 0.8091
Epoch 3/160
4263/4263 [==============================] - 1s 285us/step - loss: 0.3875 - acc: 0.8147
Epoch 4/160
4263/4263 [==============================] - 1s 281us/step - loss: 0.3853 - acc: 0.8137
Epoch 5/160
4263/4263 [==============================] - 1s 286us/step - loss: 0.3785 - acc: 0.8201
Epoch 6/160
4263/4263 [==============================] - 1s 279us/step - loss: 0.3808 - acc: 0.8178
Epoch 7/160
4263/4263 [==============================] - 1s 288us/step - loss: 0.3762 - acc: 0.8198
Epoch 8/160
4263/4263 [==============================] - 1s 288us/step - loss: 0.3749 - acc: 0.8211
Epoch 9/160
4263/4263 [==============================] - 1s 291us/step - loss: 0.3715 - acc: 0.8221
Epoch 10/160
4263/4263 [==============================] - 1s 291us/step - loss: 0.3690 - acc: 0.82

4263/4263 [==============================] - 1s 299us/step - loss: 0.3186 - acc: 0.8493
Epoch 83/160
4263/4263 [==============================] - 1s 296us/step - loss: 0.3132 - acc: 0.8513
Epoch 84/160
4263/4263 [==============================] - 2s 362us/step - loss: 0.3141 - acc: 0.8504
Epoch 85/160
4263/4263 [==============================] - 1s 286us/step - loss: 0.3153 - acc: 0.8519
Epoch 86/160
4263/4263 [==============================] - 1s 346us/step - loss: 0.3151 - acc: 0.8537
Epoch 87/160
4263/4263 [==============================] - 1s 321us/step - loss: 0.3146 - acc: 0.8533
Epoch 88/160
4263/4263 [==============================] - 1s 270us/step - loss: 0.3145 - acc: 0.8524
Epoch 89/160
4263/4263 [==============================] - 1s 263us/step - loss: 0.3106 - acc: 0.8523
Epoch 90/160
4263/4263 [==============================] - 1s 264us/step - loss: 0.3122 - acc: 0.8538
Epoch 91/160
4263/4263 [==============================] - 1s 262us/step - loss: 0.3081 - acc: 0.8556
Epo

In [23]:
import flask
import numpy as np
#import tensorflow as tf
from keras.models import load_model

app = flask.Flask(__name__)
def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('model/speed_detect.h5')
    graph = tf.get_default_graph()
    
def getParameters():
    parameters = []
    parameters.append(flask.request.args.get('Wifi_density'))
    parameters.append(flask.request.args.get('Intersection_density'))
    parameters.append(flask.request.args.get('Honk_duration'))
    parameters.append(flask.request.args.get('Timelevel'))
    parameters.append(flask.request.args.get('Road_surface'))
    
    return parameters

def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

@app.route("/predict", methods=["GET"])
def predict():
    
    parameters = getParameters()
    inputFeature = np.asarray(parameters).reshape(1,5)
    with graph.as_default():
        raw_prediction = model.predict(inputFeature)
    print(raw_prediction)
   
   # k_1=round(raw_prediction[0])
    k_1_i=int(raw_prediction[0][0])
        #k_1_s=str(k_1_i)
    #k_2=round(raw_prediction[1])
    k_2_i=int(raw_prediction[0][1])
        #k_2_s=str(k_2_i)
    #k_3=round(raw_prediction[2])
    k_3_i=int(raw_prediction[0][2])
        #k_3_s=str(k_3_i)
   # k_4=round(raw_prediction[3])
    k_4_i=int(raw_prediction[0][3])
        #k_4_s=str(k_4_i)
        
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        prediction='Bad Data'
        
    else:
        if k_1_i==1:
            prediction='Fast'
        if k_2_i==1:
            prediction='Normal'
        if k_3_i==1:
            prediction='Slow'
        if k_4_i==1:
            prediction='Very Fast'
    print(prediction)
        
    return sendResponse({prediction})


if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
"please wait until server has fully started"))
    init()
    app.run(threaded=True)

* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2019-05-27 19:05:34,185] ERROR in app: Exception on /predict [GET]
Traceback (most recent call last):
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 2292, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1815, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1718, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/_compat.py", line 35, in reraise
    raise value
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 1813, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/abhijit/.local/lib/python3.6/site-packages/flask/app.py", line 17

[[0. 1. 0. 0.]]
Normal


In [24]:
import flask
import numpy as np
#import tensorflow as tf
from keras.models import load_model
from json import dumps, loads, JSONEncoder, JSONDecoder
import pickle

app = flask.Flask(__name__)
def init():
    global model,graph
    # load the pre-trained Keras model
    model = load_model('model/speed_detect.h5')
    graph = tf.get_default_graph()
    
def getParameters():
    parameters = []
    parameters.append(flask.request.args.get('Wifi_density'))
    parameters.append(flask.request.args.get('Intersection_density'))
    parameters.append(flask.request.args.get('Honk_duration'))
    parameters.append(flask.request.args.get('Timelevel'))
    parameters.append(flask.request.args.get('Road_surface'))
    
    return parameters

def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

@app.route("/predict", methods=["GET"])
def predict():
    
    parameters = getParameters()
    inputFeature = np.asarray(parameters).reshape(1,5)
    with graph.as_default():
        raw_prediction = model.predict(inputFeature)
    print(raw_prediction)
   
   # k_1=round(raw_prediction[0])
    k_1_i=int(raw_prediction[0][0])
        #k_1_s=str(k_1_i)
    #k_2=round(raw_prediction[1])
    k_2_i=int(raw_prediction[0][1])
        #k_2_s=str(k_2_i)
    #k_3=round(raw_prediction[2])
    k_3_i=int(raw_prediction[0][2])
        #k_3_s=str(k_3_i)
   # k_4=round(raw_prediction[3])
    k_4_i=int(raw_prediction[0][3])
        #k_4_s=str(k_4_i)
        
    if k_1_i==0 and k_2_i==0 and k_3_i==0 and k_4_i==0:
        prediction='Bad Data'
        
    else:
        if k_1_i==1:
            prediction='Fast'
        if k_2_i==1:
            prediction='Normal'
        if k_3_i==1:
            prediction='Slow'
        if k_4_i==1:
            prediction='Very Fast'
    print(prediction)
        
    return sendResponse(prediction)


if __name__ == "__main__":
    print(("* Loading Keras model and Flask starting server..."
"please wait until server has fully started"))
    init()
    app.run(threaded=True)

* Loading Keras model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/May/2019 19:09:32] "GET /predict?Wifi_density=0.0984461&Intersection_density=0.0231638&Honk_duration=1&Timelevel=3&Road_surface=4.661894235 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [27/May/2019 19:09:32] "GET /predict?Wifi_density=0.0984461&Intersection_density=0.0231638&Honk_duration=1&Timelevel=3&Road_surface=4.661894235 HTTP/1.1" 200 -


[[0. 1. 0. 0.]]
Normal


In [ ]:
from json import dumps, loads, JSONEncoder, JSONDecoder
import pickle

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (list, dict, str, unicode, int, float, bool, type(None))):
            return JSONEncoder.default(self, obj)
        return {'_python_object': pickle.dumps(obj)}
